In [20]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/samuelkim/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

# 4.4.2 NLP As A Supervised Problem
Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from Alice in Wonderland by Lewis Carroll or Persuasion by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called BoW, or Bag of Words. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

In [21]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[0:35000])
persuasion = text_cleaner(persuasion[0:35000])

In [22]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [23]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words! Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur. We will exclude stopwords and punctuation. In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [24]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [25]:
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 500


In [26]:
word_counts.head()

,out,present,quarter,australia,interesting,study,prefer,solicit,frightened,save,...,want,sadly,sweetness,1,sake,enjoyment,regret,slight,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


# Trying out BoW
Now let's give the bag of words features a whirl by trying a random forest.

In [27]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence', 'text_source'], 1)

# Take 1700 words.  This will come into play further below when we compare to Emma.
X = X.iloc[:, :1700]
X.head()

,out,present,quarter,australia,interesting,study,prefer,solicit,frightened,save,...,lip,afraid,lamp,musgrove,sleepy,1800,choose,fool,partially,distance
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9882352941176471

Test set score: 0.8771929824561403


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.

# BoW with Logistic Regression
Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [29]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(340, 1700) (340,)
Training set score: 0.9823529411764705

Test set score: 0.8728070175438597


# BoW with Gradient Boosting¶
And finally, let's see what gradient boosting can do:

In [30]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9823529411764705

Test set score: 0.8552631578947368


# Same model, new inputs
What if we feed the model a different novel by Jane Austen, like Emma? Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process Emma the same way we processed the other data, and combine it with the Alice data:

In [31]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [32]:
emma_doc = nlp(emma[0:40000])

In [33]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [34]:
emmawords = bag_of_words(emma_doc)
common_words_2 = set(emmawords + alicewords)
sentences_2 = pd.DataFrame(emma_sents + alice_sents)

word_counts_2 = bow_features(sentences_2, common_words_2)

Processing row 0
Processing row 500


In [35]:
Y_2 = word_counts_2['text_source']
X_2 = word_counts_2.drop(['text_sentence', 'text_source'], 1)
X_2 = X_2.iloc[:, :1700]

In [36]:
# Create new test set with Alice and Emma
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X_2, 
    Y_2, 
    test_size=0.4,
    random_state=42
)

print(X_test2.shape)
print(y_test2.shape)

(268, 1700)
(268,)


In [37]:
print('\nTest set score:', lr.score(X_test2, y_test2))
lr_2_predicted = lr.predict(X_test2)
pd.crosstab(y_test2, lr_2_predicted)


Test set score: 0.5186567164179104


col_0,Austen,Carroll
text_source,,
Austen,12,124
Carroll,5,127


# Drill 0

Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

In [44]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm = SVC()
svc_params = [{'C': [.00001,.0001,.001,.01,.1,1,
                    10,100,1000,10000]}]
svc_grid = GridSearchCV(svm, svc_params, cv=5, verbose=1)
train = svc_grid.fit(X_train, y_train)
print('Best Score for SVC:')
print(svc_grid.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Score for SVC:
0.85


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   15.5s finished


In [45]:
from sklearn.model_selection import cross_val_score
svm = SVC(C=10, gamma=.01)
cross_val_score(svm, X_train, y_train, cv=5)

array([0.76811594, 0.82608696, 0.88235294, 0.88059701, 0.89552239])

_The best score the model seems to generate (SVM) seems to b 0.85, slightly the below the desired accuracy of 90%._ 

# Drill 1

Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [47]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [50]:
# I pick austen-emma

austen = gutenberg.raw('austen-emma.txt')
austen = re.sub(r'VOLUME \w+', '', austen)
austen = re.sub(r'CHAPTER \w+', '', austen)
austen = text_cleaner(austen)
print(austen[:300])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence; and had lived nearly twenty-one years in the world with very little to distress or vex her. She was the youngest of the two daughters of a most affectio


In [53]:
austen_docu = nlp(austen[0:25000])

## 1.1 Alice vs. Austen

In [56]:
def punct(text):
    allwords = [token.lemma_
                for token in text
                if not token.is_stop
                ]
    return [item[0] for item in Counter(allwords).most_common(1500)]
austen_punct = punct(austen_docu)
alice_punct = punct(alice_doc)
common_words_1 = set(austen_punct + alice_punct)
austen_sents = [[sent, "Austen"] for sent in austen_docu.sents]
sents = pd.DataFrame(austen_sents + alice_sents)
word_counts = bow_features(sents, common_words)

Processing row 0
Processing row 500


In [60]:
aliceaust_word_counts = bow_features(sents, common_words_1)
aliceaust_word_counts.head()

Processing row 0
Processing row 500


,out,fortunate,realise,present,australia,interesting,frightened,save,rich,employment,...,sad,estate,tunnel,want,sadly,regret,enjoyment,slight,text_sentence,text_source
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(She, was, the, youngest, of, the, two, daught...",Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Between, _, them)",Austen


In [58]:
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence', 'text_source'], 1)
print('Scores:', cross_val_score(lr, X, Y, cv=5))
print('Avg. Score:', np.mean(cross_val_score(lr, X, Y, cv=5)))

Scores: [0.86324786 0.75213675 0.8362069  0.75862069 0.84347826]
Avg. Score 0.8107380924922154


In [76]:
#logistic regression run for Alice vs. Austen
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.3, 
                                                   random_state=0)
lr = LogisticRegression(penalty='l1')
parameters = {'C':[0.00001,0.0001,0.001,0.01,0.1,1,10,100,10000,100000,100000000]}
lr_grid = GridSearchCV(lr, parameters, cv=5, verbose=1)
lr_grid.fit(X_train, Y_train)
print('Best Score:')
print(lr_grid.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Best Score:
0.7881773399014779


[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed:    3.3s finished


## 1.2 Persuasion vs. Austen

In [68]:
persuasionwords_punct = punct(persuasion_doc)
common_words_2 = set(austen_punct + persuasionwords_punct)
sents_2 = pd.DataFrame(austen_sents + persuasion_sents)
word_counts_1 = bow_features(sents, common_words_2)
Y1 = word_counts_1['text_source']
X1 = word_counts_1.drop(['text_sentence', 'text_source'], 1)

Processing row 0
Processing row 500


In [69]:
print('Scores:', cross_val_score(lr, X1, Y1, cv=5))
print('Avg:', np.mean(cross_val_score(lr, X1, Y1, cv=5)))

Scores: [0.81196581 0.70940171 0.81896552 0.81034483 0.8173913 ]
Avg: 0.7936138341085868


In [77]:
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1, 
                                                    Y1,
                                                    test_size=0.3, 
                                                   random_state=0)
lr_grid = GridSearchCV(lr, parameters, cv=5, verbose=1)
lr_grid.fit(X1_train, Y1_train)
print('Best Score:')
print(lr_grid.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Best Score:
0.7807881773399015


[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed:    3.1s finished


_Using lasso regression for both, the model performed similarly - at 78% - for Persuasion vs. Austen and Alice vs. Austen._ 